### DESARROLLO LOCAL

In [ ]:
import pandas as pd
from google.cloud import bigquery
import gcsfs
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import sklearn
import sys

print("Versión de Python:", sys.version)
print("Versión de Pandas:", pd.__version__)
print("Versión de Numpy:", np.__version__)
print("Versión de Scikit-learn:", sklearn.__version__)
print("Versión de google-cloud-bigquery:", bigquery.__version__)
print("Version de gcsfs: ", gcsfs.__version__)

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv # package used in jupyter notebook to read the variables in file .env

""" get env variable from .env """
load_dotenv(find_dotenv())

""" Read env variables and save it as python variable """
PROJECT_ID_DS = os.environ.get("PROJECT_GCP", "")

In [ ]:
BUCKET_NAME = '{bucket-name}'

### CARGAR DATOS PKL EN STORAGE

In [ ]:
## utilizar gcsfs para utilizar GCS como si fuera local
path_data = f'gs://{BUCKET_NAME}/poc-jobs-vertex/data.pkl'

import gcsfs
import pickle
fs = gcsfs.GCSFileSystem(PROJECT_ID_DS)
with fs.open(path_data, 'rb') as f:
    data = pickle.load(f)

In [ ]:
## leer data desde pd read pickle
data = pd.read_pickle(path_data)

In [ ]:
data

### Separar en "x" "y"

In [ ]:
x = data[['feature_1', 'feature_2', 'feature_3']]
y = data[['target']]

### TRAIN MODEL

In [ ]:
model = LinearRegression()
model.fit(x,y)

In [ ]:
y_predicted = model.predict(x)

### Evaluar model

In [ ]:
r2_score(y_true = y,
         y_pred = y_predicted)

### Guardar modelo entrenado

### Guardar modelo entrenado para ser registrado en vertex models

In [ ]:
import pickle
import os
from google.cloud import storage

In [ ]:
"""
######## ESTA ES LA ÚNICA PARTE PARA TENER CUIDADO PARA QUE EL SCRIPT FUNCIONE TANTO EN UN ENTRENAMIENTO LOCAL COMO GCP ########
- Para que el modelo entrenado quede registrado en el menú "MODELS" de Vertex (artefacto del modelo obligatoriamente con el nombre "model.pkl") debe ser guardado en un bucket de GCS definido
por un path creado automático al enviar el job de entrenamiento en una variable de ambiente "AIP_MODEL_DIR" que solo existe cuando se corre el job de entrenamiento en Vertex. 

- Cuando se corre localmente la variable de ambiente no existe, por lo tanto, retorna un path: None

- Ejemplo path cloud: gs://{bucket-name}/aiplatform-custom-training-2022-05-01-13:27:34.025/model/
"""

# Save model artifact to local filesystem (doesn't persist)
artifact_filename = 'model.pkl'
local_path = artifact_filename
with open(local_path, 'wb') as model_file:
    pickle.dump(model, model_file)
print('modelo guardado local')

# Upload model artifact to Cloud Storage - try: guardar en path var de ambiente cuando se corre en cloud y registrar en vertex // except: retornar error cuando no existe path var de ambiente
path_artifact_model_vertex = f'gs://{BUCKET_NAME}/poc-jobs-vertex/modeltypeA/run_000/model/'
print('path del modelo a GCS - path_custom: ', path_artifact_model_vertex)

try:
    model_directory = path_artifact_model_vertex
    storage_path = os.path.join(model_directory, artifact_filename)
    blob = storage.blob.Blob.from_string(storage_path, client=storage.Client())
    blob.upload_from_filename(local_path)
except Exception as e: 
    print('Error: ', str(e))
    print('MODELO NO GUARDADO EN GCS')

In [ ]:
# eliminar modelo guardado en el local
os.remove('model.pkl')